# Generate a prettier plot with statistics on the plot

In [ ]:
library(repr)
library(ggpubr)
library(tidyverse)

In [ ]:
config <- list('caudate'= '../../../caudate/_m/genes/diffExpr_maleVfemale_full.txt',
               'dlpfc'= '../../../dlpfc/_m/genes/diffExpr_maleVfemale_full.txt', 
               'hippo'= '../../../hippocampus/_m/genes/diffExpr_maleVfemale_full.txt', 
               'cmc_dlpfc'= '../../../cmc_dlpfc/_m/mssm_penn_pitt_maleVfemale.tsv',
               'cmc_hbcc'= '../../../cmc_dlpfc/_m/nimh_hbcc_maleVfemale.tsv')

In [ ]:
get_deg <- function(fn){
    dft <- data.table::fread(fn)
    if('gene_id' %in% colnames(dft)){
        dft <- dft %>% 
            mutate(Feature=gene_id, Dir=sign(t)) %>% 
            rename(ensemblID=ensembl_gene_id) %>%
            select('Feature', 'ensemblID', 'adj.P.Val', 'logFC', 't', 'Dir')
    } else if('Geneid' %in% colnames(dft)) {
        dft <- dft %>% 
            mutate(Feature=Geneid, Dir=sign(t), 
                   ensemblID=str_replace(Geneid, "\\..*", "")) %>% 
            select('Feature', 'ensemblID', 'adj.P.Val', 'logFC', 't', 'Dir')
    } else {
        dft <- dft %>% 
            mutate(Feature=V1, Dir=sign(t)) %>% 
            select('Feature', 'ensemblID', 'adj.P.Val', 'logFC', 't', 'Dir')
    }
    return(dft)
}

get_deg_sig <- function(fn, fdr){
    dft <- get_deg(fn)
    return(subset(dft, adj.P.Val < fdr))
}

merge_dataframe <- function(tissue1, tissue2){
    return(merge(get_deg(config[[tissue1]]), get_deg(config[[tissue2]]), 
                 by='Feature', suffixes=c(paste0('_',tissue1), paste0('_',tissue2))))
}

merge_dataframes_sig <- function(tissue1, tissue2){
    fdr = 0.05
    return(merge(get_deg_sig(config[[tissue1]], fdr), get_deg_sig(config[[tissue2]], fdr), 
                 by='Feature', suffixes=c(paste0('_',tissue1), paste0('_',tissue2))))
}

tissue_annotation <- function(tissue){
    return(list('dlpfc'='DLPFC', 'hippo'='Hippocampus', 
                'caudate'='Caudate', 'cmc_dlpfc'='CMC DLPFC', 
                'cmc_hbcc'='CMC DLPFC: HBCC')[[tissue]])
}

get_scatter_plot <- function(tissue1, tissue2, merge_fnc, coords){
    dft <- merge_fnc(tissue1, tissue2)
    sp = ggscatter(dft, x=paste0('t_', tissue1), y=paste0('t_', tissue2), add="reg.line", 
                   xlab=paste0('T-statistic (',tissue_annotation(tissue1), ')'),
                   ylab=paste0('T-statistic (',tissue_annotation(tissue2), ')'),
                   add.params = list(color = "blue", fill = "lightgray"), conf.int = FALSE, 
                   cor.method = "pearson", cor.coef = FALSE, cor.coef.size = 7, 
                   cor.coeff.args = list(label.sep = "\n"),  #cor.coef.coord = coords, 
                   ylim=c(-125, 125)) + 
    stat_cor(aes(label = paste0(..rr.label..)), size=8, method = "pearson") + 
    font("xylab", size = 20, face='bold') + font("xy.text", size = 18)
    return(sp)
}


save_ggplots <- function(fn, p, w, h){
    for(ext in c('.pdf', '.png', '.svg')){
        ggsave(paste0(fn, ext), plot=p, width=w, height=h)
    }
}

In [ ]:
options(repr.plot.width=18, repr.plot.height=6)
sp1_sig = get_scatter_plot('caudate', 'dlpfc', merge_dataframes_sig, c(-110, 85))
sp2_sig = get_scatter_plot('caudate', 'hippo', merge_dataframes_sig, c(-110, 85))
sp3_sig = get_scatter_plot('dlpfc', 'hippo', merge_dataframes_sig, c(-110, 85))
fig1 = ggarrange(sp1_sig, sp2_sig, sp3_sig, ncol=3, align='v')
print(fig1)

In [ ]:
save_ggplots("tstatistic_corr_sig", fig1, 18, 6)

In [ ]:
sp1 = get_scatter_plot('caudate', 'dlpfc', merge_dataframe, c(-110, 85))
sp2 = get_scatter_plot('caudate', 'hippo', merge_dataframe, c(-110, 85))
sp3 = get_scatter_plot('dlpfc', 'hippo', merge_dataframe, c(-110, 85))
fig2 = ggarrange(sp1, sp2, sp3, ncol=3, align='v')
print(fig2)

In [ ]:
save_ggplots("tstatistic_corr", fig2, 18, 6)

## CommonMind comparison: MSSM Penn Pitt

In [ ]:
sp4_sig = get_scatter_plot('cmc_dlpfc', 'dlpfc', merge_dataframes_sig, c(-55, 85))
sp5_sig = get_scatter_plot('cmc_dlpfc', 'hippo', merge_dataframes_sig, c(-55, 85))
sp6_sig = get_scatter_plot('cmc_dlpfc', 'caudate', merge_dataframes_sig, c(-55, 85))
fig3 = ggarrange(sp4_sig, sp5_sig, sp6_sig, ncol=3, align='v')
print(fig3)

In [ ]:
save_ggplots("cmc_tstatistic_corr_sig", fig3, 18, 6)

In [ ]:
sp4 = get_scatter_plot('cmc_dlpfc', 'dlpfc', merge_dataframe, c(-55, 85))
sp5 = get_scatter_plot('cmc_dlpfc', 'hippo', merge_dataframe, c(-55, 85))
sp6 = get_scatter_plot('cmc_dlpfc', 'caudate', merge_dataframe, c(-55, 85))
fig4 = ggarrange(sp4, sp5, sp6, ncol=3)
print(fig4)

In [ ]:
save_ggplots("cmc_tstatistic_corr", fig4, 18, 6)

## CommonMind comparison: NIMH HBCC

In [ ]:
sp4_sig = get_scatter_plot('cmc_hbcc', 'dlpfc', merge_dataframes_sig, c(-55, 85))
sp5_sig = get_scatter_plot('cmc_hbcc', 'hippo', merge_dataframes_sig, c(-55, 85))
sp6_sig = get_scatter_plot('cmc_hbcc', 'caudate', merge_dataframes_sig, c(-55, 85))
fig3 = ggarrange(sp4_sig, sp5_sig, sp6_sig, ncol=3, align='v')
print(fig3)

In [ ]:
save_ggplots("cmc_hbcc_tstatistic_corr_sig", fig3, 18, 6)

In [ ]:
sp4 = get_scatter_plot('cmc_hbcc', 'dlpfc', merge_dataframe, c(-55, 85))
sp5 = get_scatter_plot('cmc_hbcc', 'hippo', merge_dataframe, c(-55, 85))
sp6 = get_scatter_plot('cmc_hbcc', 'caudate', merge_dataframe, c(-55, 85))
fig4 = ggarrange(sp4, sp5, sp6, ncol=3)
print(fig4)

In [ ]:
save_ggplots("cmc_hbcc_tstatistic_corr", fig4, 18, 6)

## Reproducibility Information

In [ ]:
print("Reproducibility Information:")
Sys.time()
proc.time()
options(width=120)
sessioninfo::session_info()